# 1 - Scraping Weather Data

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from urllib.error import HTTPError

In [2]:
link = "http://www.weather.gov.sg/climate-historical-daily/"
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
cities = ''
location = soup.find(class_="dropdown-menu long-dropdown").find_all('li')
for loc in location:
    cities += str(loc)
cities

'<li><a href="#Admiralty" onclick="setYear(\'S104\')">Admiralty</a></li><li><a href="#Admiralty West" onclick="setYear(\'S105\')">Admiralty West</a></li><li><a href="#Ang Mo Kio" onclick="setYear(\'S109\')">Ang Mo Kio</a></li><li><a href="#Boon Lay (East)" onclick="setYear(\'S86\')">Boon Lay (East)</a></li><li><a href="#Boon Lay (West)" onclick="setYear(\'S63\')">Boon Lay (West)</a></li><li><a href="#Botanic Garden" onclick="setYear(\'S120\')">Botanic Garden</a></li><li><a href="#Buangkok" onclick="setYear(\'S55\')">Buangkok</a></li><li><a href="#Bukit Panjang" onclick="setYear(\'S64\')">Bukit Panjang</a></li><li><a href="#Bukit Timah" onclick="setYear(\'S90\')">Bukit Timah</a></li><li><a href="#Buona Vista" onclick="setYear(\'S92\')">Buona Vista</a></li><li><a href="#Chai Chee" onclick="setYear(\'S61\')">Chai Chee</a></li><li><a href="#Changi" onclick="setYear(\'S24\')">Changi</a></li><li><a href="#Choa Chu Kang (Central)" onclick="setYear(\'S114\')">Choa Chu Kang (Central)</a></li><l

In [4]:

districts = re.findall(r'(?<=\W)[S]\d+', cities)
len(districts)

63

In [5]:
u=[]
for d in districts:
    u.append("http://www.weather.gov.sg/files/dailydata/DAILYDATA_"+ d + "_202209.csv")

In [6]:
# list of 404 pages
errlst=[]
# Check if all links are accessible 
for f in u:
    requests.get(f)
    try:
        d =pd.read_csv(f)
    except UnicodeDecodeError:
        d =pd.read_csv(f, encoding='ISO-8859-1')
    except:
        print(f)
        errlst += re.findall(r'[S]\d+', f)

errlst


http://www.weather.gov.sg/files/dailydata/DAILYDATA_S105_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S86_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S63_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S55_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S11_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S39_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S101_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S31_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S122_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S110_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S72_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S46_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S91_202209.csv


['S105',
 'S86',
 'S63',
 'S55',
 'S11',
 'S39',
 'S101',
 'S31',
 'S122',
 'S110',
 'S72',
 'S46',
 'S91']

In [7]:
ct_lst = [i for i in districts if i not in errlst]
len(ct_lst)

50

In [8]:
def get_weather(yearStart=None, yearEnd=2022):
    """ 
    input: yearStart = start year of data; yearEnd = end year of data
    output: dataframe containing weather data of all cities and months from start year to end year
    """
    
    if yearStart is None:
        raise Exception("Please indicate year range")
    else:
        months=['01','02','03','04','05','06','07','08','09','10','11','12']
        dfs=[]
        for y in range(yearStart,yearEnd+1):
            for d in ct_lst:
                for m in months:
                    
                    url = "http://www.weather.gov.sg/files/dailydata/DAILYDATA_"+ d +"_"+ str(y) + m + ".csv"
                    r = requests.get(url)

                    try:
                        df = pd.read_csv(url)
                    except UnicodeDecodeError:
                        df = pd.read_csv(url,encoding='ISO-8859-1')
                    except HTTPError as err:
                        if err.code == 302:
                            continue
                        else:
                            raise Exception("HTTP Error")
                        

                    dfs.append(df)

        res = pd.concat(dfs, ignore_index=True)

        return res

In [29]:
%%time
# tring to get all data between 2014 to 2020, run time ~ 4m55s
data = get_weather(yearStart=2014,yearEnd=2020)
data.to_csv('../data/weather_data.csv')

CPU times: user 35 s, sys: 6.77 s, total: 41.8 s
Wall time: 4min 55s


In [30]:
df = pd.read_csv('weather_data.csv')
df.head()

/var/folders/tg/8mpzp58s7wd9h25zt2bffqd00000gn/T/ipykernel_868/2326829862.py:1: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('weather_data.csv')


,Unnamed: 0,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm)
0,0,Admiralty,2014.0,1.0,1.0,0.0,0.0,0.0,0.0,26.3,28.8,24.3,10.8,34.2,NaN,NaN,NaN
1,1,Admiralty,2014.0,1.0,2.0,0.0,0.0,0.0,0.0,26.9,30.7,24.6,11.5,38.2,NaN,NaN,NaN
2,2,Admiralty,2014.0,1.0,3.0,4.6,4.4,4.4,4.6,26.5,29.5,24.9,9.7,34.2,NaN,NaN,NaN
3,3,Admiralty,2014.0,1.0,4.0,3.8,3.8,3.8,3.8,26.4,31.2,24.7,8.1,32.8,NaN,NaN,NaN
4,4,Admiralty,2014.0,1.0,5.0,3.6,1.2,1.6,2.0,24.6,25.3,23.6,8.4,27.0,NaN,NaN,NaN


# 2- Scraping Historical Dengue Data

In [31]:
# data taken from https://outbreak.sgcharts.com/data 
import pandas as pd
import glob
import os

path = r'/Users/natasha/Documents/GA/ClassMaterial_Edit/DSIF-SG-7/project_4_dengue_case/data/dengue_case' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=None)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [32]:
frame.head()

,0,1,2,3,4,5,6,7,8
0,1,pasir ris street 71 (block 747),1.379194,103.934693,1,1,2,170320,3
1,1,pasir ris street 72 (whitewater),1.380042,103.935474,1,1,2,170320,3
2,2,bunga rampai place,1.338931,103.883537,1,1,32,150925,9
3,9,joo seng road (block 8),1.335219,103.878805,1,1,32,150925,9
4,3,mount vernon road,1.340377,103.879490,1,1,32,150925,9


In [33]:
frame.rename(columns={0: 'no_case', 1: 'address',2:'lattitude',3:'longitude',4:'cluster_no',5:'recent_case_cluster',6:'total_case_cluster',7:'date',8:'month'}, inplace=True)
frame.to_csv('../data/dengue_case.csv')
